In [54]:
import pandas as pd
import json
import os
path = r'E:\Processed'
paths =[]
for i in os.listdir(path):
    paths.append(path + '\\' +i)
paths

['E:\\Processed\\Amazon Fashion',
 'E:\\Processed\\Appliances1',
 'E:\\Processed\\Arts & Crafts',
 'E:\\Processed\\CD & Vinyl',
 'E:\\Processed\\Cell phones',
 'E:\\Processed\\Clothing shoes jwellery',
 'E:\\Processed\\Digital Music',
 'E:\\Processed\\Industrial and Scientific',
 'E:\\Processed\\Kindle Store',
 'E:\\Processed\\Luxury Beauty',
 'E:\\Processed\\Movies & TV',
 'E:\\Processed\\Musical Instruments',
 'E:\\Processed\\Office Products',
 'E:\\Processed\\Patio, Lawn & Garden',
 'E:\\Processed\\Pet Supplies',
 'E:\\Processed\\Toys & Games',
 'E:\\Processed\\Video Games']

In [55]:
for path in paths:
    csv_path = path+'\\classified.csv'
    json_path = path + '\\scraped_j.json'
    to_save = path +'\\classified_updated.csv'
    #Read metadata
    print('reading csv(s)')
    metadata =  pd.read_csv(csv_path)

    #Scraped json
    # read brands full
    with open(json_path,'r') as f:
        a = json.loads(f.read())
    cats = []
    for i in a:
        cats.append(list(i.keys())[0])
    print('stemming the titles')

    #lemmatize metedata title column
    import re
    from nltk.stem.snowball import SnowballStemmer
    stemmer = SnowballStemmer(language='english')

    #lemmatizer = WordNetLemmatizer()
    def lem(x):
        x = re.sub(r'[^\w\s]', '', str(x)) 
        x = x.split()
        y = []
        for i in x:
            y.append(stemmer.stem(i.lower()))
        return " ".join(y)
    metadata.title = metadata.title.apply(lambda x: lem(x))

    #getting vocabulary out
    print('building vocabulary')
    corpus = []
    def vocab(x):
        a = x[1:].split('/')[-1]
        if a.count('&') ==1: #just for the ones with one '&'
            y = a.replace(',','').replace("'","").split('&')

            if all([len(i.split()) == 1 for i in y]): #NOT for a b & c d format
                for i in y:
                    corpus.append(stemmer.stem(i))

            else: #for a b & c d format
                corpus.append(stemmer.stem(a))
        elif a.count('&') >1 :
            pass

        else:
            corpus.append(stemmer.stem(a))

        b = x[1:].split('/')[:-1]
        for i in b:
            corpus.append(stemmer.stem(i))
    a = pd.Series(cats)
    a.apply(lambda x: vocab(x))
    print('Generating mapping words')

    #mapping words
    mw = pd.DataFrame(corpus)[0].value_counts()[pd.DataFrame(corpus)[0].value_counts() == 1]
    mw = mw.index

    #sorting according to the number of words
    mw = pd.DataFrame(mw)
    mw[1] = mw[0].apply(lambda x: len(x.split()))
    mw = mw.loc[mw[1].sort_values().index]
    mw = mw[0]
    mw = mw.apply(lambda x :x.lower())

    #Lemmatize cat map
    df_k = pd.DataFrame(cats,columns = ['category_unlemmetize'])

    def lem(x):
        #x = re.sub(r'[^\w\s]', '', str(x)) 
        x = x.split('/')[1:]
        y = []
        for i in x:
            y.append(stemmer.stem(i.lower()))
        return "/".join(y)
    print('stemming category')

    df_k['category'] = df_k.category_unlemmetize.apply(lambda x: lem(x))

    #Extract mapping keys from category map
    def key_ex(x):
        y = []
        for i in mw:
            if i in x[1:].lower().split('/'):
                y.append(i)
        return y
    df_k['keys'] = df_k.category.apply(lambda x: key_ex(x))
    #The ones where keys isnt present
    df_catm = df_k[df_k['keys'].apply(lambda x: str(x) == '[]')]
    #where keys is present
    df_km = df_k[df_k['keys'].apply(lambda x: str(x) != '[]')]


    def matchsingle(title,key): # Also does the "or" form of matching if contains &
        x = title
        y = key # Condition should be len(y) == 1
        if not bool(re.search('&',y[0])):
            y = y[0].replace(',','').replace("'","").split()
            if all(stemmer.stem(i.strip()) in x.split() for i in y):
                return True
            return False
        else: #doing a or b search in a & b
            y1 = y[0].replace(',','').replace("'","").split('&')
            if all([len(i.split()) == 1 for i in y1]):
                for i in y1:
                    if matchsingle(x,[stemmer.stem(i)]):
                        return True
                return False
            else: # if string like a b & c d; it will match a, b or c, d
                y[0] = y[0].replace(',','').replace("'","")
                #print(y[0])
                try:
                    span = re.search('\w+ & \w+',y[0]).span()
                    a1 = y[0][span[0]:span[-1]]
                    a2 = y[0][:span[0]]+' '+y[0][span[-1]:]
                    a = matchsingle(x,[stemmer.stem(a1)])
                    b = matchsingle(x,[' '.join(stemmer.stem(j) for j in a2.split())])
                    if all([a,b]):
                        return True
                    else:
                        return False
                except Exception as e:
                    return False
    def matchmultiple(title,key):
        for i in key:
            if matchsingle(title,[i]):
                return True
        return False

    df_km.drop(df_km[df_km.category_unlemmetize == '/'].index,inplace = True)

    def matcher(x):
        for i in df_km.values:
            if len(i[1]) == 1:
                if matchsingle(x,i[2]):
                    return i[0]
            else:
                if matchmultiple(x,i[2]):
                    return i[0]
        return []
    
    print('matching keys')

    a = metadata.title.apply(lambda x: matcher(x))
    a1 = a[a.apply(lambda x: str(x)!='[]')]
    print(len(a1))
    a1
    metadata.category = a

    df_catm['keys'] = df_catm.category.apply(lambda x: x.split('/')[-2:])
    df_catm['keys_len'] = df_catm['keys'].apply(lambda x: len(' '.join(x).split()))
    df_catm = df_catm.set_index('keys_len')
    df_catm = df_catm.sort_index()
    df_catm

    import numpy 
    def catmMatcher(x):

        for i in df_catm.values:
            b = []
            a = i[0]
            for j in i[2]:
                #print(j)
                b.append(matchsingle(x,[j]))
            if numpy.prod(b):
                return a
        return []

    metadata.loc[a[a.apply(lambda x: str(x)=='[]')].index,:]
    print('matching categories')
    b = metadata.loc[a[a.apply(lambda x: str(x)=='[]')].index,:].title.apply(lambda x: catmMatcher(x))
    b2 = b[b.apply(lambda x :str(x)!='[]')]
    metadata.loc[a[a.apply(lambda x: str(x)=='[]')].index,'category'] = b
    z = metadata.loc[a[a.apply(lambda x: str(x)=='[]')].index,:].category
    classified = metadata[metadata.category.apply(lambda x: str(x) != '[]')]

    print("saving csv's")
    classified.to_csv(to_save.split('.')[0]+'new.csv')


    # In[31]:


    classified#/to_csv(to_save)
    df = pd.read_csv(csv_path)
    df.set_index('asin',inplace = True)
    df = df.drop(classified.asin)
    df = df.reset_index()
    df.to_csv(to_save.split('.')[0]+'check.csv')

print('*******************************'+path+'done**************************')

reading csv(s)
stemming the titles
building vocabulary
Generating mapping words
stemming category
matching keys


C:\Users\shubh\Anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


KeyboardInterrupt: 